In [1]:
import numpy as np
import pandas as pd
import sympy as sp

from utilities import SingleDetector

In [2]:
SLOPE_ANGLE = 3 * sp.pi / 360
DETECTOR_ANGLE = 2 * sp.pi / 3
DIRECTION_ANGLE = sp.pi/2

PRECISION = 20

In [3]:
# 创建一个 dataframe 用于存储数据
# 四列分别为 dis_from_center, water_depth, scan_width, ocerlap_ratio
df = pd.DataFrame(columns=['dis_from_center', 'water_depth', 'scan_width', 'ocerlap_ratio'])
# dis_from_center 值从 -800 到 800，步长为 200
df['dis_from_center'] = np.arange(-800, 1000, 200)

# water_depth 值在 dis_from_center 为 0 时为 70，按照步长递增
df['water_depth'] = 70 - df['dis_from_center'] * sp.tan(SLOPE_ANGLE)
df['water_depth'] = df['water_depth'].apply(lambda x: x.evalf(PRECISION))
df

,dis_from_center,water_depth,scan_width,ocerlap_ratio
0,-800,90.948737255349543111,NaN,NaN
1,-600,85.711552941512157334,NaN,NaN
2,-400,80.474368627674771556,NaN,NaN
3,-200,75.237184313837385778,NaN,NaN
4,0,70.000000000000000000,NaN,NaN
5,200,64.762815686162614222,NaN,NaN
6,400,59.525631372325228444,NaN,NaN
7,600,54.288447058487842666,NaN,NaN
8,800,49.051262744650456889,NaN,NaN


In [4]:
detectors = []

for i in range(len(df)):
    # 创建一个 PointLoc 对象
    detector = SingleDetector(center_depth=df['water_depth'][i], 
                              slope_angle=SLOPE_ANGLE,
                              detector_angle=DETECTOR_ANGLE,
                              direction_angle=DIRECTION_ANGLE,
                              precision=PRECISION)
    detector.get_scan_segment()
    detectors.append(detector)
    # 计算扫描宽度
    df.loc[i, 'scan_width'] = detector.scan_segment_length_proj_xy

    if i > 0:
        # 计算重叠率
        slope_length_between_origin = (df.loc[i, 'dis_from_center'] - df.loc[i-1, 'dis_from_center']) / sp.cos(SLOPE_ANGLE)
        slope_length_half_pre = sp.Segment(detectors[i-1].origin, detectors[i-1].detect_point_1).length.evalf(PRECISION)
        slope_length_half_cur = sp.Segment(detectors[i].origin, detectors[i].detect_point_2).length.evalf(PRECISION)
        df.loc[i, 'ocerlap_ratio'] = ((slope_length_half_pre + slope_length_half_cur - slope_length_between_origin) / slope_length_between_origin).evalf(PRECISION)

df

,dis_from_center,water_depth,scan_width,ocerlap_ratio
0,-800,90.948737255349543111,315.70510695333724903,NaN
1,-600,85.711552941512157334,297.52557105398511351,0.53101534713891183217
2,-400,80.474368627674771556,279.34603515463297798,0.44011766764215115456
3,-200,75.237184313837385778,261.16649925528084246,0.34921998814539047694
4,0,70.000000000000000000,242.98696335592870694,0.25832230864862979933
5,200,64.762815686162614222,224.80742745657657142,0.16742462915186912171
6,400,59.525631372325228444,206.62789155722443587,0.076526949655108444042
7,600,54.288447058487842666,188.44835565787230035,-0.014370729841652233607
8,800,49.051262744650456889,170.26881975852016483,-0.10526840933841291122
